In [1]:
import warnings
warnings.filterwarnings('ignore')
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func



In [2]:
engine = create_engine("sqlite:///hawaii.sqlite",echo=False)

In [3]:
engine.execute('SELECT * FROM measurements LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [4]:
Base = automap_base()
Base.prepare(engine,reflect=True)
Base.classes.keys()





['measurements', 'stations']

In [5]:
Measurement = Base.classes.measurements
Station = Base.classes.stations



In [6]:
session = Session(engine)

## Design a query to retrieve the last 12 months of precipitation data.



In [7]:
from sqlalchemy import inspect, func, desc, extract, select
import datetime
from dateutil import parser
import pandas as pd
import numpy as np

inspector = inspect(engine)
columns = inspector.get_columns('measurements')
for column in columns:
    print(column['name'], column['type'])




id INTEGER
station TEXT
date DATE
prcp FLOAT
tobs FLOAT


In [8]:
engine.execute('SELECT * FROM measurements LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [9]:
sel = [Measurement.station, Measurement.date, Measurement.prcp]
annual_rainfall = session.query(*sel).order_by(Measurement.date.desc()).all()
annual_rainfall_df = pd.DataFrame(np.array(annual_rainfall).reshape(len(annual_rainfall),3), 
                                  columns = ['STATION','DATE', 'RAIN'])
annual_rainfall_df.head(10)
       
       
       

,STATION,DATE,RAIN
0,USC00519397,2017-08-23,0
1,USC00514830,2017-08-23,0
2,USC00519523,2017-08-23,0.08
3,USC00516128,2017-08-23,0.45
4,USC00519397,2017-08-22,0
5,USC00519523,2017-08-22,0
6,USC00516128,2017-08-22,0.5
7,USC00519397,2017-08-21,0
8,USC00514830,2017-08-21,0.02
9,USC00519523,2017-08-21,0


In [10]:
last12_months = session.query(Measurement.station, Measurement.date, Measurement.prcp).filter(Measurement.date >= '2016-08-23').\
    order_by(Measurement.date).all()
last12_months_df = pd.DataFrame(last12_months)
last12_months_df.head()


,station,date,prcp
0,USC00519397,2016-08-23,0.00
1,USC00513117,2016-08-23,0.15
2,USC00514830,2016-08-23,0.05
3,USC00517948,2016-08-23,0.00
4,USC00519523,2016-08-23,0.02


## PLOT DATES VS. PRECIPITATION

In [11]:
import cmocean
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.axes import Subplot
import matplotlib.ticker as mtick
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools




In [12]:
x = last12_months_df.date
y = last12_months_df.prcp
station = last12_months_df.station

cmap = cmocean.cm.ice

trace1 = go.Bar(
    x=x, 
    y=y, 
    name= "Rain by Date"
)

marker = dict(
    cmap=cmap,
    line=dict(colorscale=cmap, width=1.5), 
    hoverlabel=True,
    text = last12_months_df.station,
    hoverinfo='text',
    opacity=0.6
)
  
data = [trace1] 
layout = go.Layout(
    title="Precipitation Analysis",
        titlefont=dict(
            family='Raleway',
            size=18
        ),
    xaxis=dict(
        tickangle=-45,
        title="Date",
            titlefont=dict(
                family='Raleway',
                size=16
            ),
        showgrid=True,
        ),
    yaxis=dict(
        title='Rainfall',
            titlefont=dict(
                family='Raleway',
                size=16
            ),
        showgrid=True
    ),
    showlegend=True,
    legend=dict(
        x=x, 
        y=y, 
        traceorder='normal',
        font=dict(
            family='Raleway', 
            size=14
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)   

In [13]:
last12_months_df.describe()


,prcp
count,2230.000000
mean,0.160664
std,0.442067
min,0.000000
25%,0.000000
50%,0.010000
75%,0.110000
max,6.700000


## STATION ANALYSIS

In [14]:
totalStations = session.query(Measurement.station).distinct().count()

In [15]:
session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).\
    order_by(desc(func.count(Measurement.tobs))).all()

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [16]:
most_active = session.query(Measurement.station, func.count(Measurement.tobs)).\
                group_by(Measurement.station).\
                order_by(desc(func.count(Measurement.tobs))).first()


In [17]:
most_active

('USC00519281', 2772)

In [18]:
details_most_active = session.query(Measurement.date, Measurement.tobs).\
        filter(Measurement.date > '2016-08-23', Measurement.station == most_active.station).all()
details_most_active_df = pd.DataFrame(details_most_active)
details_most_active_df.head()


,date,tobs
0,2016-08-24,77.0
1,2016-08-25,80.0
2,2016-08-26,80.0
3,2016-08-27,75.0
4,2016-08-28,73.0


In [19]:
x0 = details_most_active_df.tobs
x1 = details_most_active_df.date

trace1 = go.Histogram(
    x=x0,
    name='TObs',
    nbinsx=12,
    xbins=dict(
        start=55,
        end=85,
        size=2.5
    ),
    marker=dict(
        color='rgba(171, 50, 96, 0.6)'
    ),
    opacity=0.75
)

data = [trace1]

layout = go.Layout(
    title='Station Analysis',
    titlefont=dict(
                family='Raleway',
                size=18
            ),
    xaxis=dict(
        title='Value',
        showgrid=True,
        titlefont=dict(
                family='Raleway',
                size=16
            )
    ),
    yaxis=dict(
        title='Frequency',
        showgrid=True,
        titlefont=dict(
                family='Raleway',
                size=16
            )
    ),
    bargap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## TEMPERATURE ANALYSIS

In [20]:
def calc_temps (start_date, end_date, plot=True):
    temp = session.query(func.avg(Measurement.tobs),
                         func.min(Measurement.tobs).label('min_temp'), 
                         func.max(Measurement.tobs).label('avg_temp')).\
                            filter((Measurement.date >= start_date) & (Measurement.date <= end_date)).all()[0]        
    print("Min Temp = %2.2f,  Avg Temp = %2.2f,  Max Temp = %2.2f" \
        % (temp[1], temp[0], temp[2]))
    trace = go.Bar(
        y=temp[0],
        width = 0.25,
        name="Average Temps",
        marker = dict(
            color = 'hsl(358, 78%, 47%)',
        ),
        error_y = dict(
            type = 'constant',
            value = list(temp),
            thickness = 3,
            color = 'hsl(312, 6%, 33%)'
        ),
        )
    
    data= [trace]
    
    layout = go.Layout(
        title = "<b>Trip Average Temp</b>",
        titlefont = dict(
            family = 'Raleway',
            size = 18,
        ),
        yaxis = dict(title = 'Temperature'),
        xaxis = dict(title = '{} to {}'.format(start_date, end_date),
                    showticklabels=False),
        )
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig)
    
  


In [21]:
calc_temps('2015-01-24', '2016-01-24')

Min Temp = 56.00,  Avg Temp = 74.11,  Max Temp = 86.00


## CREATE FLASK APPLICATION